# **Getting them stonks: Portfolio mean-variance optimaization**
---

## Contents:

>[1 - Introduction](#1---Introduction)
>
>[2 - Importing modules](#2---Importing-modules)
>
>[3 - Data retrieval](#3---Data-retrieval)
>
>[4 - Preprocessing](#4---Preprocessing)
>
>[5 - Modelling](#5---Modelling)
>
>[6 - Conclusion](#6---Conclusion)
>


## 1 - Introduction

1. Present aim
2. Explain Theory (mean-variance optimization)
3. Get data (summary stats)
4. Plot Efficient frontier + portfolio points (MC simulation)
5. Change parameters
6. Extensions

### **Q:** Situation: You won the lottery, recieved the paycheck for your summer internship, or that distant uncle you didn't even know passed and left you some money... what do you do?
### **A:** Invest... but how?

Recepie for investment:

1. Define a goal/strategy
2. Pick suitable assets
3. **Construct a suitable portfolio**
4. Check and repeat

### **Q:** Given $n$ assets, what is the optimal allocation of these within a portfolio?
### **A:** There are many...

### The Mean-Variance framework:
- Developed by Harry Markowitz in 1952 (earned him Nobel Price in Economics)
- Aims to solve the above problem using two ingredients:
    1. The volatility of asset returns (risk) - for stocks, this is the average log first difference in stock prices
    2. The expected asset returns (reward) - for stocks, this is the sample covariance of periodic returns
- Shortcomings:
    - Stock returns can be non-stationary $\implies$ we can't used average returns as a reasonable forecast
    - Stock returns are notoriously hard to forecast (Efficient Market Hypothesis)
    
### Goal: Using those two ingredients, find an set of weights for how much each asset should make up of the total portfolio
    

## 2 - Importing modules

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
%matplotlib inline

## 3 - Data retrieval

First, we need to gather data on stock prices for a selection of assets. We focus our attention on [Investopedia's Top Stocks for March 2021](https://www.investopedia.com/top-stocks-4581225) during the period Feb 2018 - Feb 2021.

In [2]:
# Specify asset symbols
stocks = ['NRG','BIO','VIRT','WTM','ALL','MAT','FCX','IAC','ZM','CE','MRNA','PTON','ETSY','TSLA','ZS']
data = web.DataReader(stocks, 'yahoo', start='2019/02/10', end='2021/02/10')
data.head()

Attributes  Adj Close                                                \
Symbols           NRG         BIO       VIRT         WTM        ALL   
Date                                                                  
2019-02-11  40.368961  252.929993  23.600658  920.919434  88.779411   
2019-02-12  40.521839  258.100006  23.862984  913.227234  89.144028   
2019-02-13  40.655602  261.790009  23.763483  901.424561  90.189827   
2019-02-14  40.712940  261.420013  23.265959  898.381592  89.451050   
2019-02-15  40.569614  270.339996  22.858894  903.908752  90.554420   

Attributes                                           ...      Volume  \
Symbols           MAT        FCX IAC  ZM         CE  ...         MAT   
Date                                                 ...               
2019-02-11  15.740000  11.329431 NaN NaN  91.073799  ...  16303600.0   
2019-02-12  16.469999  11.290126 NaN NaN  94.205116  ...  15181600.0   
2019-02-13  17.070000  12.076210 NaN NaN  93.768631  ...  10584100.0   
2019-02-14  16.910000  11.948471 NaN NaN  94.451813  ...   6968400.0   
2019-02-15  13.820000  12.066383 NaN NaN  95.653938  ...  33526900.0   

Attributes                                                            \
Symbols            FCX IAC  ZM         CE       MRNA PTON       ETSY   
Date                                                                   
2019-02-11  15561600.0 NaN NaN  1123000.0   597900.0  NaN  2122300.0   
2019-02-12  15617900.0 NaN NaN  1511900.0   753100.0  NaN  2576600.0   
2019-02-13  36169400.0 NaN NaN   811700.0  1333500.0  NaN  2073500.0   
2019-02-14  15313400.0 NaN NaN   921500.0  1091400.0  NaN  1595400.0   
2019-02-15  16573400.0 NaN NaN  1337400.0  2087400.0  NaN  2537500.0   

Attributes                         
Symbols           TSLA         ZS  
Date                               
2019-02-11  35648500.0  1122800.0  
2019-02-12  27588000.0  1493600.0  
2019-02-13  25708000.0  1194800.0  
2019-02-14  26004000.0  1315000.0  
2019-02-15  19524500.0  1018100.0  

[5 rows x 90 columns]

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 505 entries, 2019-02-11 to 2021-02-10
Data columns (total 90 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (Adj Close, NRG)   505 non-null    float64
 1   (Adj Close, BIO)   505 non-null    float64
 2   (Adj Close, VIRT)  505 non-null    float64
 3   (Adj Close, WTM)   505 non-null    float64
 4   (Adj Close, ALL)   505 non-null    float64
 5   (Adj Close, MAT)   505 non-null    float64
 6   (Adj Close, FCX)   505 non-null    float64
 7   (Adj Close, IAC)   155 non-null    float64
 8   (Adj Close, ZM)    458 non-null    float64
 9   (Adj Close, CE)    505 non-null    float64
 10  (Adj Close, MRNA)  505 non-null    float64
 11  (Adj Close, PTON)  347 non-null    float64
 12  (Adj Close, ETSY)  505 non-null    float64
 13  (Adj Close, TSLA)  505 non-null    float64
 14  (Adj Close, ZS)    505 non-null    float64
 15  (Close, NRG)       505 non-null    float64
 16  (Close,

## 4 - Preprocessing

In [4]:
data = data['Adj Close']
data.head()

Symbols,NRG,BIO,VIRT,WTM,ALL,MAT,FCX,IAC,ZM,CE,MRNA,PTON,ETSY,TSLA,ZS
Date,,,,,,,,,,,,,,,
2019-02-11,40.368961,252.929993,23.600658,920.919434,88.779411,15.740000,11.329431,NaN,NaN,91.073799,18.170000,NaN,53.150002,62.568001,48.590000
2019-02-12,40.521839,258.100006,23.862984,913.227234,89.144028,16.469999,11.290126,NaN,NaN,94.205116,18.690001,NaN,55.830002,62.362000,50.099998
2019-02-13,40.655602,261.790009,23.763483,901.424561,90.189827,17.070000,12.076210,NaN,NaN,93.768631,18.530001,NaN,55.040001,61.633999,49.000000
2019-02-14,40.712940,261.420013,23.265959,898.381592,89.451050,16.910000,11.948471,NaN,NaN,94.451813,19.660000,NaN,54.060001,60.754002,50.080002
2019-02-15,40.569614,270.339996,22.858894,903.908752,90.554420,13.820000,12.066383,NaN,NaN,95.653938,21.440001,NaN,54.660000,61.576000,50.110001


In [5]:
returns = (np.log(data)).diff()
returns.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 505 entries, 2019-02-11 to 2021-02-10
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NRG     504 non-null    float64
 1   BIO     504 non-null    float64
 2   VIRT    504 non-null    float64
 3   WTM     504 non-null    float64
 4   ALL     504 non-null    float64
 5   MAT     504 non-null    float64
 6   FCX     504 non-null    float64
 7   IAC     154 non-null    float64
 8   ZM      457 non-null    float64
 9   CE      504 non-null    float64
 10  MRNA    504 non-null    float64
 11  PTON    346 non-null    float64
 12  ETSY    504 non-null    float64
 13  TSLA    504 non-null    float64
 14  ZS      504 non-null    float64
dtypes: float64(15)
memory usage: 63.1 KB


In [6]:
ex_returns = returns.mean()
cov_returns = returns.cov()

## 5 - Modelling

1. Define our objective function
2. Find gradient vector
3. Implement Stochastic Gradient Descent
4. OPTIMIZE!

#### Mathematically:
#### $ \underset{W}{\text{min}} \quad  W^T\:\Sigma \: W \quad \textrm{s.t}\quad  R^T\!W = \mu \;, \quad \sum_{i=1}^{n}{w_i}=1$
#### where:

#### $W=\begin{bmatrix}
w_1\\
\vdots \\
w_n
\end{bmatrix} \quad,\quad R=\begin{bmatrix}
\mathbb{E}[r_1]\\
\vdots \\
\mathbb{E}[r_n]
\end{bmatrix} \quad,\quad\Sigma = \begin{bmatrix}
\sigma_{11} & \dots & \sigma_{n1}\\
\vdots & \ddots & \vdots\\
\sigma_{1n} & \dots & \sigma_{nn}
\end{bmatrix}$

#### $ \underset{W}{\text{min}} \quad  W^T\:\Sigma \: W \quad \textrm{s.t}\quad  R^T\!W = \mu \;, \quad \sum_{i=1}^{n}{w_i}=1$
#### $\implies \mathcal{L}(W,\lambda) = W^T\:\Sigma \: W - \lambda( R^T\!W - \mu)$

In [7]:
def objective(w):
    cov_returns


Approximating the gradient numerically using **finite differences method**

* Backward difference  $f'(x) \approx \frac{f(x_k) - f(x_k - \epsilon)}{\epsilon}$
* Forward difference  $f'(x) \approx \frac{f(x_k + \epsilon) - f(x_k)}{\epsilon}$
* Central difference $f'(x) \approx \frac{f(x_k + \frac{\epsilon}{2}) - f(x_k - \frac{\epsilon}{2})}{\epsilon}$

The central difference approximation gives the most accurate one among these three. Therefore, let's implement that one here.

In [8]:
def central_finite_diff(f, x):
    dim = x.shape[0]
    
    eps  = np.sqrt(np.finfo(float).eps) 
    grad = np.zeros((1,dim))
    
    for i in range(dim):
        e = np.zeros((1,dim))
        e[0,i] = eps
        grad_approx = (f(x + (e/2)) - f(x-(e/2)))/eps
        grad[0,i] = grad_approxr
    return grad


In [9]:
def gradient_descent(f, x0, grad_f, lr, max_iter=1e5, grad_tol=1e-4, traj=False):
    '''
    Gradient Descent
    INPUTS:
        f        : Function
        x0       : Initial guess
        grad_f   : Gradient function
        lr       : Learning rate
        max_iter : Maximum number of iterations
        grad_tol : Tolerance for gradient approximation
        traj     : Boolean for plotting
    OUTPUTS:
        x        : Optimal point
        iter_i   : Number of iterations needed
    '''
    
    # Initialize problem
    x      = np.copy(x0)
    iter_i = 0
    grad_i = grad_tol*10
    
    # Plotting
    if traj == True:
        x_list = []
        f_list = []
        
    while np.sum(np.abs(grad_i)) > grad_tol and iter_i < max_iter:
        
        grad_i  = grad_f(f, x) # compute gradient
        x       = x - lr*grad_i       # compute step
        iter_i += 1
        
        # Plotting
        if traj == True:
            x_list.append(x.flatten().tolist())
            f_list.append(f(x))
        
    print(' Optimization using Gradient Descent \n')
    print('Iterations: ', iter_i)
    print('Optimal x : ', x) 
    print('Final grad: ', grad_i)
    
    # Trajectory    
    if traj == True:
        return x, x_list, f_list, 
        
    return x, iter_i

## 6 - Conclusion
